In [2]:
# Import Python Tools:
from IPython.display import display, Latex
import pandas as pd
import numpy as np
from numpy import sqrt, pi, average, cos, sin, tan, arcsin, arccos, arctan, deg2rad, rad2deg

import load_combos
from steel_design.AISC import aisc
from masonry_design.steel_reinforcement import rebar_df as rebar
from concrete_design.material_properties import elastic_modulus, rigidity_modulus
import masonry_design.material_properties as block
conc_density = 150.0 #pcf
steel_density = 0.2833 #pci

display(Latex(r"\newpage")) #pagebreak

<IPython.core.display.Latex object>

# Steel Beam - Increase Moment of Inertia
## Bar Sizing

In [12]:
# list(aisc.columns.values)
beam_exst = "W12X19"
I_existing = aisc.loc[beam_exst, "Ix"]
I_existing #in^4

130.0

In [10]:
I_target = aisc.loc["W14X22", "Ix"]
I_target #in^4

199.0

In [16]:
I_min_add = I_target - I_existing
I_min_add

69.0

In [11]:
rebar

,Diameter,Area,Weight
#3,0.375,0.11,0.376
#4,0.500,0.20,0.668
#5,0.625,0.31,1.043
#6,0.750,0.44,1.502
#7,0.875,0.60,2.044
#8,1.000,0.79,2.670
#9,1.128,1.00,3.400
#10,1.270,1.27,4.303
#11,1.410,1.56,5.313
#14,1.693,2.25,7.650


In [54]:
bar_size = "#8"
A_bar = rebar.loc[bar_size, "Area"]
d_bar = rebar.loc[bar_size, "Diameter"]
I_bar = pi*d_bar**4/64 #in^4

# Beam geometry
d = aisc.loc[beam_exst, "d"]
tf = aisc.loc[beam_exst, "tf"]
bf = aisc.loc[beam_exst, "bf"]
k_det = aisc.loc[beam_exst, "kdet"]
bar_attachment_space = (bf - k_det) / 2
print(f"space to fit bar under flange = {bar_attachment_space} in")

d_bar = (d - 2*tf - d_bar) / 2 #in

I_add = 2 * (2*I_bar + 2*A_bar*d_bar**2) #top and bottom bars (4 total bars added)
f"Added I = {round(I_add, 2)} in^4"

space to fit bar under flange = 1.5675 in


'Added I = 87.29 in^4'

In [55]:
I_total_new = I_existing + I_add
f"Total Moment of Inertia = {round(I_total_new, 2)} in^4"

'Total Moment of Inertia = 217.29 in^4'

In [56]:
I_total_new > I_target

True

## Weld Design
### Shear Flow Calc

In [57]:
Fy = 50.0 #ksi
V = 0.6 * Fy * aisc.loc[beam_exst, "A"] #kips
print(f"V = {round(V, 2)} kips")

yi = tf / 2 #in
Ai = tf * bf #in^2
Q = yi * Ai #in^3

I = I_total_new #in^4

q = V * Q / I * 1000 #lbs/in
f"q = {round(q, 2)} lbs/in"

V = 167.1 kips


'q = 188.88 lbs/in'

In [58]:
q_weld = q / 2 #lbs/in
f"q_weld = {round(q_weld, 2)} lbs/in"

'q_weld = 94.44 lbs/in'

### Welds
#### Weld Metal

In [1]:
phi = 0.75
F_EXX = 70.0 #ksi
Fnw = 0.60*F_EXX

R = 3/16 #in
t = 5/16 * R #in

spacing = 36.0 #in O.C.
lw = 2.0 #in
Awe = t * lw #in^2

Rn = Fnw * Awe
phiRn = phi * Rn
print(f"{round(phiRn, 2)} kips")
f"phiRn / in = {round(phiRn / spacing * 1000, 2)} lbs/in"

3.69 kips


'phiRn / in = 102.54 lbs/in'

In [76]:
phiRn / spacing * 1000 > q_weld

True

#### Base Metal

In [64]:
phi = 1.00
F_nBM = 0.6 * Fy
A_BM = tf * lw

Rn = F_nBM * A_BM
phiRn = phi * Rn
f"phiRn = {round(phiRn, 2)} kips"

'phiRn = 21.0 kips'